# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [26]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [4]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = int(len(newsgroups_test.target) / 2)
test_data, test_labels = newsgroups_test.data[num_test:], newsgroups_test.target[num_test:]
dev_data, dev_labels = newsgroups_test.data[:num_test], newsgroups_test.target[:num_test]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('test label shape:', test_labels.shape)
print('dev label shape:', dev_labels.shape)
print('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


### Part 1:

For each of the first 5 training examples, print the text of the message along with the label.

In [5]:
def P1(num_examples=5):
    for i in range(num_examples):
        print ('Label' + ':', newsgroups_train.target_names[train_labels[i]]), 
        print (train_data[i])
        print (" ")
        print ("/////////NEXT/////////")
        print (" ")
P1()

Label: comp.graphics
Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
 
/////////NEXT/////////
 
Label: talk.religion.misc


Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating such evil corruption
fo

### Part 2:

Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [59]:
def P2():
    
    print ("a:")
    vectorizer = CountVectorizer()
    train_counts_default = vectorizer.fit_transform(train_data)
    print ('Size of the vocabulary:', len(vectorizer.vocabulary_))
    num = (train_counts_default.nnz)/2034
    print ("Average non-zero features per example: {:0.2f}.\n".format(num))
    print ("Fraction of non zero entries from training data:", (train_counts_default.nnz)/(26879.0*2034))
    
    print ("//////")
    
    print ("b:")
    print ("First Feature String:",vectorizer.get_feature_names()[0])
    print ("Last Feature String:", vectorizer.get_feature_names()[26878])
           
    print ("//////")
    
   
    print ("c:")
    vectorizer_c = CountVectorizer(vocabulary=["atheism", "graphics", "space", "religion"])
    train_counts_custom_vocab = vectorizer_c.transform(train_data)
    print ('Shape with custom vocabulary: ', train_counts_custom_vocab.shape)
    print ('Avg number of Non zero features with custom vocabulary: ',float(train_counts_custom_vocab.nnz)/2034.0)
    
    print ("//////")
    

    print ("d:")
    vectorizer_d = CountVectorizer(analyzer='char', ngram_range=(2,2))
    train_counts_bigram = vectorizer_d.fit_transform(train_data)
    print ('Size of Vocabulary for bigram data:' , len(vectorizer_d.vocabulary_))
    
    vectorizer_e = CountVectorizer(analyzer='char', ngram_range=(3,3))
    train_counts_trigram = vectorizer_d.fit_transform(train_data)
    print ('Size of Vocabulary for trigram data:' , len(vectorizer_d.vocabulary_))
    
    print ("//////")
    
   
    print ("e:")
    vectorizer_f = CountVectorizer(min_df=10)
    train_counts_min10 = vectorizer_f.fit_transform(train_data)
    print ('Size of vocab with min_df=10: ', len(vectorizer_f.vocabulary_))
    
    print ("//////")
    
  
    print ("f:")
    vectorizer_h = CountVectorizer()
    train_counts_dev = vectorizer_h.fit_transform(dev_data)
    print ('Size of vocab with dev data: ', len(vectorizer_h.vocabulary_))
    print ('Fraction of words missing in dev data: ', float(len(vectorizer.vocabulary_)-len(vectorizer_h.vocabulary_))/float(26879))
    

P2()

a:
Size of the vocabulary: 26879
Average non-zero features per example: 96.71.

Fraction of non zero entries from training data: 0.0035978272269590263
//////
b:
First Feature String: 00
Last Feature String: zyxel
//////
c:
Shape with custom vocabulary:  (2034, 4)
Avg number of Non zero features with custom vocabulary:  0.26843657817109146
//////
d:
Size of Vocabulary for bigram data: 3291
Size of Vocabulary for trigram data: 3291
//////
e:
Size of vocab with min_df=10:  3064
//////
f:
Size of vocab with dev data:  16246
Fraction of words missing in dev data:  0.395587633468507


### Part 3:

Use the default CountVectorizer options and report the f1 score (use metrics.f1_score with average="weighted") for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

* Why doesn't nearest neighbors work well for this problem?
* Any ideas why logistic regression doesn't work as well as Naive Bayes?
* Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [72]:
def P3():

    count_vectorizer = CountVectorizer()
    train_counts = count_vectorizer.fit_transform(train_data)
    dev_count = count_vectorizer.transform(dev_data)

   
    knn_clf = KNeighborsClassifier()
    knn_params = {'n_neighbors':[1, 3, 5, 7, 9, 15, 25]}
    gsv_knn = GridSearchCV(knn_clf,knn_params,scoring='f1_weighted')
    gsv_knn.fit(train_counts,train_labels)
    print ("best k value:")
    print (gsv_knn.best_params_)
    
   
    mnb_clf = MultinomialNB()
    mnb_params = {'alpha':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
    gsv_mnb = GridSearchCV(mnb_clf,mnb_params,scoring='f1_weighted')
    gsv_mnb.fit(train_counts,train_labels)
    print ("Best alpha value:")
    print (gsv_mnb.best_params_)

    
    l_reg_params = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.5, 2.0, 5.0, 10.0, 25.0, 50.0, 100.0]
    scores = np.empty((17, ))
    
    
    cnt = 0;
    for l in l_reg_params:
        l_reg_clf = LogisticRegression(C=l) 
        model = l_reg_clf.fit(train_counts,train_labels)
        
      
        predicted_labels_lr = l_reg_clf.predict(dev_count)
        scores[cnt]=metrics.f1_score(dev_labels,predicted_labels_lr,average='weighted')
      
        print ('C:' + str(l)+ ': ', np.sum(np.square(model.coef_[0])), "," ,
               np.sum(np.square(model.coef_[1])), ",", np.sum(np.square(model.coef_[2])), "," ,
               np.sum(np.square(model.coef_[3])))
        cnt+=1
        
    print (" ")
    score = np.where(scores==np.max(scores))[0] 
    print ('Best C value:')
    print(l_reg_params[score[0]])
   
P3()

best k value:
{'n_neighbors': 7}


/Users/candice/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
/Users/candice/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
/Users/candice/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
/Users/candice/opt/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:511: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '
/Users/candice/opt/anaconda3/lib/python3.8/site-packages/skl

Best alpha value:
{'alpha': 0.1}


/Users/candice/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C:0.1:  14.128206384116098 , 14.492792306267589 , 15.517147411493088 , 12.891563679350764


...

/Users/candice/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C:25.0:  325.58526764240804 , 268.23800774943504 , 306.64867120256054 , 316.53823595277896


/Users/candice/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


C:50.0:  394.5925060258676 , 330.9833123707083 , 376.8688865071427 , 386.44763806453585
C:100.0:  482.9040340388268 , 405.33301140546706 , 465.1122095737507 , 483.7332644251226
 
Best C value:
0.1


/Users/candice/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ANSWER: K-nearest neighbors doesn't work well for this problem due to overfitting on training data. Logistic regression doesn't work well for the same reason of overfitting but it works a little bit better than KNN. 
C val increases when the sum of squared weughts increase because when the C is higher, the effect of regularization decreases.

### Part 4:

Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [75]:
def P4():
    labels = ["Unigram", "Bigram"]
    for n in range(2):
        vectorizer = CountVectorizer(ngram_range=(n+1, n+1))
        d_matrix_train = vectorizer.fit_transform(train_data)
        d_matrix_dev = vectorizer.transform(dev_data)
      
    
        model = LogisticRegression()
        model.fit(d_matrix_train, train_labels)
     
    
        sorted_wt = np.argsort(model.coef_, axis=1)
        
        maxindex_wt = sorted_wt[:, -5:].flatten()    
        
        words = np.array(vectorizer.get_feature_names())
        
        print (labels[n] + ":")
        
        print (words[maxindex_wt])
        print (np.transpose(np.round(model.coef_[:, maxindex_wt],2)))
        
        print ("Score:")
        print (model.score(d_matrix_dev, dev_labels))
        
        print (" ")
P4()

/Users/candice/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Unigram:
['posting' 'atheism' 'religion' 'atheists' 'deletion' 'files' '3d' 'file'
 'image' 'graphics' 'spacecraft' 'launch' 'nasa' 'orbit' 'space' 'fbi'
 'blood' 'god' 'christian' 'christians']
[[ 0.63 -0.13 -0.06 -0.45]
 [ 0.68 -0.29 -0.27 -0.12]
 [ 0.74 -0.4  -0.49  0.14]
 [ 0.75 -0.06 -0.21 -0.49]
 [ 0.95 -0.33 -0.34 -0.27]
 [-0.3   0.65 -0.03 -0.32]
 [-0.17  0.75 -0.38 -0.2 ]
 [-0.15  0.92 -0.41 -0.36]
 [-0.33  1.02 -0.44 -0.25]
 [-0.36  1.4  -0.69 -0.35]
 [-0.18 -0.24  0.64 -0.22]
 [-0.26 -0.21  0.65 -0.18]
 [-0.42 -0.07  0.79 -0.3 ]
 [-0.19 -0.33  0.85 -0.32]
 [-0.59 -0.46  1.67 -0.62]
 [-0.18 -0.17 -0.24  0.6 ]
 [-0.37 -0.09 -0.23  0.68]
 [ 0.35 -0.55 -0.55  0.76]
 [-0.32 -0.28 -0.17  0.77]
 [-0.34 -0.22 -0.3   0.86]]
Score:
0.6923076923076923
 
Bigram:
['you are' 'look up' 'was just' 'cheers kent' 'claim that' 'out there'
 'is there' 'in advance' 'comp graphics' 'looking for' 'sci space'
 'how ironic' 'sherzer methodology' 'the moon' 'the space' 'cheers kent'
 'is strength' 'i

### Part 5:

Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

ANSWER: It is interesting that bigram accuracy score is less than unigram, and also it is easier to make connections in unigram.

In [87]:
import string

def empty_preprocessor(s):
    return s

def better_preprocessor(s):
    s = s.lower()
    s = re.sub('(\d)+', 'numb', s)
    
    tb = str.maketrans({key: None for key in string.punctuation})
    s = s.translate(tb)
    s = re.sub("\s+", " ", s)
    s = s.strip(" ")
    return s

def fScore(model, dev):
    predict = model.predict(dev)
    fScore = metrics.f1_score(dev_labels, predict, average='weighted')
    return fScore

def P5():
    vectorizer = CountVectorizer(preprocessor = empty_preprocessor)
    
    x = vectorizer.fit_transform(train_data).toarray()
    dev = vectorizer.transform(dev_data).toarray()
    model = LogisticRegression(penalty = 'l2', solver='lbfgs', multi_class='multinomial', max_iter=1000)
    model.fit(x, train_labels)
    
    fScore = CalcFScore(model, dev)
    w_len = len(vectorizer.vocabulary_)
    print ("Empty Preprocessor:")
    print ("Size of dictionary:")
    print (w_len)
    print("F1 Score:")
    print (fScore)
    print (" ")
    vectorizer = CountVectorizer(preprocessor = better_preprocessor)
    
    x = vectorizer.fit_transform(train_data).toarray()
    dev = vectorizer.transform(dev_data).toarray()
    model = LogisticRegression(penalty = 'l2', solver='lbfgs', multi_class='multinomial', max_iter=1000)
    model.fit(x, train_labels)
    
    fScore = CalcFScore(model, dev)
    w_len_impr = len(vectorizer.vocabulary_)
    print ("Better Preprocessor:")
    print ("Size of dictionary:")
    print (w_len_impr)
    print("F1 Score:")
    print (fScore)
    
    d_words = w_len - w_len_impr
    print("Better preprocessor reduced the vocab size by: {}".format(d_words))
P5()

Empty Preprocessor:
Size of dictionary:
33291
F1 Score:
0.6952528365499843
 
Better Preprocessor:
Size of dictionary:
27224
F1 Score:
0.673563907826199
Better preprocessor reduced the vocab size by: 6067


### Part 6:

The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.015 (the default is .0001).

In [71]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)
     
    count_vectorizer = CountVectorizer()
    train_count = count_vectorizer.fit_transform(train_data)
    dev_count = count_vectorizer.transform(dev_data)
    
    #L1
    lr_clf_l1 = LogisticRegression(penalty='l1', solver='saga', multi_class='multinomial', max_iter=1000, C = 1.0, tol=0.01)
    lr_cl_l1 =  lr_clf_l1.fit(train_count, train_labels)
    
    #L2
    lr_clf_l2 = LogisticRegression(penalty='l2', solver='saga', multi_class='multinomial', max_iter=1000, C = 1.0, tol=0.01)
    lr_clf_l2 =  lr_clf_l2.fit(train_count, train_labels)
    
    print ('The number of nonzero weights with L1: ', np.count_nonzero(lr_clf_l1.coef_))
    print ('The number of nonzero weights with L2: ',np.count_nonzero(lr_clf_l2.coef_))
    
P6()

The number of nonzero weights with L1:  22523
The number of nonzero weights with L2:  107516


### Part 7:

Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [90]:
def P7():
    vectorizer = TfidfVectorizer()
    train = vectorizer.fit_transform(train_data)
    dev = vectorizer.transform(dev_data)
    
    
    model = LogisticRegression(penalty = 'l2', solver='saga', multi_class='multinomial', max_iter=1000, C = 100, tol=0.01)
    model.fit(train, train_labels)
    predict = model.predict(dev)
    probs = model.predict_proba(dev)
    
    r = []
    for i, row in enumerate(probs):
        prob_max = max(row)
        prob_correct = row[dev_labels[i]]
        r.append(prob_max/prob_correct)
        
    top_3 = sorted(range(len(r)), key=lambda i: r[i])[-3:]
    
    print("Top 3 documents with largest R ratio:")
    for i in top_3:
        print("\nR-score: {:.4}\n".format(r[i]))
        print(dev_data[i])
        print("//////////////////////////////////////////")
P7()

Top 3 documents with largest R ratio:

R-score: 244.6


Well, you were going well until you hit this one.

Hawaii was an independent country.  A coup by Americans led to a request to
annex it.  The US refused, but eventually did annex it several years later
during the Spanish-American War.
--
"On the first day after Christmas my truelove served to me...  Leftover Turkey!
On the second day after Christmas my truelove served to me...  Turkey Casserole
    that she made from Leftover Turkey.
[days 3-4 deleted] ...  Flaming Turkey Wings! ...
   -- Pizza Hut commercial (and M*tlu/A*gic bait)
//////////////////////////////////////////

R-score: 475.0

I am pleased to announce that a *revised version* of _The Easy-to-Read Book
of Mormon_ (former title: _Mormon's Book_) by Lynn Matthews Anderson is now
available through anonymous ftp (see information below). In addition to the
change in title, the revised ETR BOM has been shortened by several pages
(eliminating many extraneous "that's" and "of

ANSWER: TfidtVectorizer converts the documenr into a tfidf matrix scores as it applies an transform to the data. Countvectorizer counts the num of times a word occurs in the doc, and converts it into a count matrix.

 

### Part 8 EXTRA CREDIT:

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.